# Importation des bibliothéques

In [1]:
import pandas as pd
import glob

In [2]:
import pandas as pd
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# for dataframe and udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *


from pyspark.sql.functions import col
from pyspark.sql import functions as F

from igraph import Graph
from pyspark.sql.window import Window
from pyspark.sql.functions import col

from functools import reduce
from pyspark.sql.functions import collect_set, min as min_, expr,array_min,max
from pyspark.sql.functions import lit

from pyspark.sql.types import StringType, IntegerType, LongType, FloatType, ArrayType,StructType,StructField, BooleanType

import pandas as pd
import glob

import time

import matplotlib.pyplot as plt

from pyspark.sql.types import BooleanType


In [3]:
import os
os.environ["SPARK_HOME"] =  "/root/anaconda3/lib/python3.9/site-packages/pyspark" 
os.environ["JAVA_HOME"] ="/usr"

# Lancer spark

In [4]:
findspark.init() 

def demarrer_spark(NB_CORES):
  local = f"local[{NB_CORES}]"
  # le parametre spark.local.dir indique le repertoire contenant les données temporaires ecrites sur disque lorsque le shufle ne tient pas en memoire
  appName = "PLDAC"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "320G").\
  set("spark.driver.memory","320G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.driver.maxResultSize", "20G").\
  set("spark.local.dir", "/data/bd/spark/tmp").\
  set("spark.sql.execution.arrow.pyspark.enabled", "true")
    

# # to allow sharing in memory arow format between pandas and spark : speeds up the creation of a spark df from a pandas df
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")


  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  # spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")
    
  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (NB_CORES coeurs)
  shuffle_partitions = 3 * NB_CORES
  print("shuffle", shuffle_partitions)
  spark.conf.set("spark.sql.shuffle.partitions", str(shuffle_partitions))    

  print("session démarrée, son id est ", sc.applicationId)
  return spark


# Lecture des données

In [5]:
spark = demarrer_spark(60)


24/04/30 17:20:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/30 17:20:35 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/04/30 17:20:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/30 17:20:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


shuffle 180
session démarrée, son id est  local-1714490436485


In [6]:
parquet_folder = "/data/bd/dataset/proteine/80_80/G99/graph_deduplicated_v2/"

df = spark.read.parquet(parquet_folder)
df.show(20)

+-----------+------------+-----------+-------------+------------+----------------+-------+------------+---+
|   query_id|query_length|  target_id|target_length|match_length|percent_identity|e_value|relative_sim|  r|
+-----------+------------+-----------+-------------+------------+----------------+-------+------------+---+
|100003172:0|          89|143094166:3|          799|          89|           100.0|2.2E-46|         1.0|  1|
|100003668:0|         102| 48815973:0|          222|         102|            99.0|3.9E-55|        0.99|  1|
|100004301:0|         169|106189136:2|          695|         169|            99.4|3.9E-94|       0.994|  1|
|100004301:0|         169|117840121:3|          264|         169|           100.0|1.9E-94|         1.0|  1|
|100004301:0|         169| 44038038:1|          524|         169|           100.0|5.4E-94|         1.0|  1|
|100004301:0|         169| 49847952:5|          458|         169|            99.4|1.2E-93|       0.994|  1|
|100004301:0|         169| 6

In [7]:
df = df[['target_id', 'query_id']]
df.count()

341807741

Ici dans chaque dossier ayant le nom N_partitions : contient N partitions : 

In [ ]:
# Répartissez le DataFrame en n partitions
nb_partitions = [200] # ici on peut mettre une liste si pn souhaote avoir plusieurs dossiers avec différents nombre de partitionnemnts
for num_partitions in nb_partitions :
    df_repartitioned = df.repartition(num_partitions, 'target_id')

    # Écrivez le DataFrame réparti sur le disque
    # Vous pouvez ajuster le format et l'emplacement de sortie selon vos besoins
    output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions"
    df_repartitioned.write.mode("overwrite").parquet(output_path)


## Pour renommer les fichiers afin de les adapter au format de nos méthodes :

Chaque fichier aura un nom sous le formart : partition_NUM_DE_LA_PARTITION.parquet
et ces fichiers ont contenus dans un dossier nommé N_partitions comme précisé précedemment

In [9]:
import os
num_partitions = 200
# Chemin vers le dossier contenant les fichiers à renommer
output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions"

# Liste de tous les fichiers non cachés dans le dossier
files = [file for file in os.listdir(output_path) if not (file.startswith('.') or file=="_SUCCESS")]  # Exclure les fichiers cachés
print(len(files))

# Numéro de départ pour le renommage
start_index = 0

# Parcourir tous les fichiers
for filename in files:
    # Construction du nouveau nom de fichier en utilisant le numéro actuel
    new_filename = f"partition_{start_index}.parquet"
    
    # Chemin complet vers l'ancien et le nouveau fichier
    old_file_path = os.path.join(output_path, filename)
    new_file_path = os.path.join(output_path, new_filename)
    
    # Renommer le fichier
    os.rename(old_file_path, new_file_path)
    
    # Affichage pour confirmation
    print(f"Renommage de {filename} en {new_filename}")
    
    # Incrémenter le numéro de départ pour le prochain fichier
    start_index += 1


200
Renommage de part-00181-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_0.parquet
Renommage de part-00122-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_1.parquet
Renommage de part-00155-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_2.parquet
Renommage de part-00043-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_3.parquet
Renommage de part-00150-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_4.parquet
Renommage de part-00042-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_5.parquet
Renommage de part-00140-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_6.parquet
Renommage de part-00099-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_7.parquet
Renommage de part-00011-ae90d12e-6ff5-48f9-ab84-3595289fde18-c000.snappy.parquet en partition_8.parquet
Renommage de part-00100-ae90d12e-6ff5-48f9-ab84-3595289fde18

### Quelques tests : 

In [16]:
num_partitions = 50
output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions/part-00049-bbae4cde-d0d9-4958-9a0a-5953aa180d0c-c000.snappy.parquet"

df = spark.read.parquet(output_path)
num_rows = df.count()
print(f"Le fichier {output_path} contient {num_rows} lignes.")


Le fichier /data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/50_partitions/part-00049-bbae4cde-d0d9-4958-9a0a-5953aa180d0c-c000.snappy.parquet contient 6904102 lignes.


In [6]:
num_partitions = 150
output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions/part-00149-cf23f7e6-df01-44ab-a44f-c9faa14d4ab4-c000.snappy.parquet"

df = spark.read.parquet(output_path)
num_rows = df.count()
print(f"Le fichier {output_path} contient {num_rows} lignes.")


Le fichier /data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/150_partitions/part-00149-cf23f7e6-df01-44ab-a44f-c9faa14d4ab4-c000.snappy.parquet contient 2305246 lignes.


In [34]:
spark.stop()